In [2]:
%pip install duckdb

  Using cached duckdb-1.3.0-cp312-cp312-win_amd64.whl.metadata (7.2 kB)
Using cached duckdb-1.3.0-cp312-cp312-win_amd64.whl (11.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import duckdb
print("DuckDB imported successfully!")

DuckDB imported successfully!


In [7]:
import os
print(os.getcwd())

c:\Users\Pardis\AppData\Local\Programs\Microsoft VS Code


In [22]:
import pandas as pd
import duckdb

import os
print(os.getcwd())

df = pd.read_csv(r"C:\Users\Pardis\OneDrive\Desktop\sales_data_02.csv",skiprows=1)



c:\Users\Pardis\AppData\Local\Programs\Microsoft VS Code


In [14]:
print(df.columns)


Index(['Row Labels', 'East', 'North', 'South', 'West', 'Grand Total'], dtype='object')


In [18]:
df.columns = ['Category', 'East', 'North', 'South', 'West', 'Total']


In [19]:
df.head()

,Category,East,North,South,West,Total
0,Accessories,450.0,500,NaN,NaN,950
1,Electronics,NaN,4000,5000.0,1400.0,10400
2,Grand Total,450.0,4500,5000.0,1400.0,11350


In [20]:
df_melted = df.melt(id_vars='Category', 
                    value_vars=['East', 'North', 'South', 'West'],
                    var_name='Region',
                    value_name='Revenue')

df_melted = df_melted[df_melted['Category'] != 'Grand Total']


In [21]:
import duckdb

result = duckdb.query("""
    SELECT Category, Region, SUM(Revenue) AS Total_Revenue
    FROM df_melted
    GROUP BY Category, Region
""").to_df()

print(result)

      Category Region  Total_Revenue
0  Electronics   East            NaN
1  Electronics  North         4000.0
2  Accessories  South            NaN
3  Accessories   West            NaN
4  Electronics  South         5000.0
5  Electronics   West         1400.0
6  Accessories   East          450.0
7  Accessories  North          500.0


In [25]:
print(df.columns)

Index(['Row Labels', 'East', 'North', 'South', 'West', 'Grand Total'], dtype='object')


In [27]:
print(df.head())

    Row Labels   East  North   South    West  Grand Total
0  Accessories  450.0    500     NaN     NaN          950
1  Electronics    NaN   4000  5000.0  1400.0        10400
2  Grand Total  450.0   4500  5000.0  1400.0        11350


In [28]:
df_long = df.melt(id_vars=['Row Labels'], value_vars=['East', 'North', 'South', 'West'], 
                  var_name='Region', value_name='Total_Revenue')

In [29]:
import duckdb

df_long['Total_Revenue'] = df_long['Total_Revenue'].fillna(0)

con = duckdb.connect()
con.register('df_view', df_long)

result = con.execute("""
    SELECT "Row Labels" AS Category, Region, SUM(Total_Revenue) AS Revenue
    FROM df_view
    GROUP BY "Row Labels", Region
""").fetchdf()

print(result)

       Category Region  Revenue
0   Electronics  South   5000.0
1   Electronics   West   1400.0
2   Electronics   East      0.0
3   Electronics  North   4000.0
4   Accessories   East    450.0
5   Accessories  North    500.0
6   Grand Total  South   5000.0
7   Grand Total   West   1400.0
8   Accessories  South      0.0
9   Accessories   West      0.0
10  Grand Total   East    450.0
11  Grand Total  North   4500.0


In [33]:
df_long = df.melt(id_vars=['Row Labels'], value_vars=['East', 'North', 'South', 'West'],
                  var_name='Region', value_name='Total_Revenue')

df_long['Total_Revenue'] = df_long['Total_Revenue'].fillna(0)


In [35]:
result = con.execute("""
SELECT "Row Labels" AS Category, SUM(Total_Revenue) AS Total_Revenue
FROM df_long
GROUP BY "Row Labels"
ORDER BY Total_Revenue DESC
                     """).fetchdf()
print(result)


      Category  Total_Revenue
0  Grand Total        11350.0
1  Electronics        10400.0
2  Accessories          950.0


In [36]:
result = con.execute("""
SELECT Region, SUM(Total_Revenue) AS Total_Revenue
FROM df_long
GROUP BY Region
ORDER BY Total_Revenue DESC
                     """).fetchdf()
print(result)

  Region  Total_Revenue
0  South        10000.0
1  North         9000.0
2   West         2800.0
3   East          900.0


In [37]:
result = con.execute("""
SELECT "Row Labels" AS Category, Region, Total_Revenue
FROM df_long
WHERE Total_Revenue = (
    SELECT MAX(Total_Revenue)
    FROM df_long AS sub
    WHERE sub.Region = df_long.Region
)
ORDER BY Region

                     """).fetchdf()
print(result)

      Category Region  Total_Revenue
0  Accessories   East          450.0
1  Grand Total   East          450.0
2  Grand Total  North         4500.0
3  Electronics  South         5000.0
4  Grand Total  South         5000.0
5  Electronics   West         1400.0
6  Grand Total   West         1400.0


In [38]:
result = con.execute("""
SELECT "Row Labels" AS Category, AVG(Total_Revenue) AS Avg_Revenue
FROM df_long
GROUP BY "Row Labels"
ORDER BY Avg_Revenue DESC
                     """).fetchdf()
print(result)

      Category  Avg_Revenue
0  Grand Total       2837.5
1  Electronics       2600.0
2  Accessories        237.5


In [39]:
result = con.execute("""
SELECT "Row Labels" AS Category, Total_Revenue
FROM df_long
WHERE Region = 'East'
ORDER BY Total_Revenue DESC
LIMIT 3
                     """).fetchdf()
print(result)

      Category  Total_Revenue
0  Grand Total          450.0
1  Accessories          450.0
2  Electronics            0.0
